In [49]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import parsing_contructs as my
import time
from backup import backup
from backup import now
import os
import random
from parsing_contructs import get_contruct44
from parsing_contructs import get_contruct233
from parsing_contructs import get_empty_dataframe 
from parsing_contructs import get_win_info_44
from parsing_contructs import get_win_info_233

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.options.display.max_columns = 100
%matplotlib inline

# workbook

### Извлекаем файл:

In [4]:
directory = r'C:\Users\v.mazeiko\Documents\Парсинг Госзаупок\Input_data'
os.listdir(directory)

['OrderSearch_07.06.2019.xlsx',
 'Мурманск - результаты поиска.xlsx',
 'РезультатыПоиска-29-05-2019.xlsx']

In [5]:
file = directory + '\\' + os.listdir(directory)[0]
data = pd.ExcelFile(file)
data.sheet_names

['44-отходы', '223-отходы ', '44-лампы', '223-лампы', 'tmp', 'data2', 'data1']

In [6]:
data = data.parse(data.sheet_names[-2])

In [7]:
data.head(2)

,Реестровый номер закупки,Закупки по,"Способ определения поставщика, подрядной организации (размещения закупки)",Наименование закупки,type,Начальная (максимальная) цена контракта,Наименование Заказчика,Дата размещения,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Идентификационный код закупки
0,№0349100004618000225,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, вывозу и обезврежи...",med,515100.0,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,Нет ограничений,2018-12-27,2019-01-21,'181511200012851120100100052180000000'
1,№0349100004618000224,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, транспортированию ...",med,879840.0,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,"Субъектам малого предпринимательства, социальн...",2018-12-27,2019-01-21,'181511200012851120100100052160000000'


### Подгототавливаем столбец с номерами закупок к работе. Функция не воспринимает знак №

In [8]:
data['Реестровый номер закупки'] = data['Реестровый номер закупки'].str.replace('№','')

In [9]:
data.head()

,Реестровый номер закупки,Закупки по,"Способ определения поставщика, подрядной организации (размещения закупки)",Наименование закупки,type,Начальная (максимальная) цена контракта,Наименование Заказчика,Дата размещения,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Идентификационный код закупки
0,0349100004618000225,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, вывозу и обезврежи...",med,515100.00,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,Нет ограничений,2018-12-27,2019-01-21,'181511200012851120100100052180000000'
1,0349100004618000224,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, транспортированию ...",med,879840.00,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,"Субъектам малого предпринимательства, социальн...",2018-12-27,2019-01-21,'181511200012851120100100052160000000'
2,0349100004618000228,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по транспортированию и размещен...,med,49183.20,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-28,Нет ограничений,2018-12-28,2019-01-23,'181511200012851120100100052170000000'
3,0149200002318006192,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по транспортированию и обезвреж...,med,1115000.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-12-25,Нет ограничений,2018-12-25,2019-01-11,'182519080011451900100100662030000000'
4,0349100021118000118,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по вывозу и утилизации медицинс...,med,1086758.87,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-19,"Субъектам малого предпринимательства, социальн...",2018-12-19,2018-12-28,'181519005315951900100101181163821000'


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 12 columns):
Реестровый номер закупки                                                     168 non-null object
Закупки по                                                                   168 non-null object
Способ определения поставщика, подрядной организации (размещения закупки)    168 non-null object
Наименование закупки                                                         168 non-null object
type                                                                         168 non-null object
Начальная (максимальная) цена контракта                                      168 non-null float64
Наименование Заказчика                                                       168 non-null object
Дата размещения                                                              168 non-null datetime64[ns]
Особенности размещения заказа\ Преимущества (44-ФЗ)                          168 non-null object
Дата начала п

In [11]:
data.columns.values

array(['Реестровый номер закупки', 'Закупки по',
       'Способ определения поставщика, подрядной организации (размещения закупки)',
       'Наименование закупки', 'type',
       'Начальная (максимальная) цена контракта',
       'Наименование Заказчика', 'Дата размещения',
       'Особенности размещения заказа\\ Преимущества (44-ФЗ)',
       'Дата начала подачи заявок', 'Дата окончания подачи заявок',
       'Идентификационный код закупки'], dtype=object)

###  Выделяем список закупок по фз44

In [12]:
orders44 = data.loc[(data['Закупки по'] == '44-ФЗ')]['Реестровый номер закупки']
orders44.head()

0    0349100004618000225
1    0349100004618000224
2    0349100004618000228
3    0149200002318006192
4    0349100021118000118
Name: Реестровый номер закупки, dtype: object

### Создаём пустой датафрейм и наполняем его спарсенными данными.

### по фз44:

In [13]:
df = get_empty_dataframe()
df

,Реестровый номер закупки,победитель,цена контракта после снижения,номер контракта


In [14]:
i = 1
for order in orders44:
    tmpdf = get_win_info_44(order)
    if tmpdf is not None:
        df = pd.concat([df, tmpdf])
        print(f'{order} - ok! {i} of {len(orders44)} is done')
    else:
        print(f'{order} - checked. {i} of {len(orders44)} is done')
    i += 1
    time.sleep(1)

0349100004618000225 - ok! 1 of 159 is done
0349100004618000224 - ok! 2 of 159 is done
0349100004618000228 - ok! 3 of 159 is done
0149200002318006192 - checked. 4 of 159 is done
0349100021118000118 - ok! 5 of 159 is done
0149200002318005842 - ok! 6 of 159 is done
0349200022018000059 - ok! 7 of 159 is done
0149200002318005863 - ok! 8 of 159 is done
0149200002318005050 - ok! 9 of 159 is done
0149200002318005280 - ok! 10 of 159 is done
0149200002318005233 - ok! 11 of 159 is done
0149200002318005238 - ok! 12 of 159 is done
0149200002318005091 - ok! 13 of 159 is done
0149200002318005081 - ok! 14 of 159 is done
0149200002318004994 - ok! 15 of 159 is done
0149200002318005000 - ok! 16 of 159 is done
0349200017118000031 - ok! 17 of 159 is done
0149200002318004967 - checked. 18 of 159 is done
0149200002318004616 - ok! 19 of 159 is done
0149200002318004712 - ok! 20 of 159 is done
0349100016718000176 - ok! 21 of 159 is done
0149200002318004516 - ok! 22 of 159 is done
0149200002318004502 - ok! 23 of

In [15]:
df.head()

,Реестровый номер закупки,победитель,цена контракта после снижения,номер контракта
0,0349100004618000225,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СОРЭ...",512524.50,1511200012819000036
0,0349100004618000224,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",875440.80,1511200012819000042
0,0349100004618000228,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",49183.20,1511200012819000029
0,0349100021118000118,"Общество с ограниченной ответственностью ""Сорэ...",516210.92,1519005315919000016
0,0149200002318005842,ООО 'СорэксМед',572882.00,2519214003019000009


### по фз223:

In [16]:
orders223 = data.loc[(data['Закупки по'] == '223-ФЗ')]['Реестровый номер закупки']
orders223.head()

155    31807326700
156    31807270424
157    31807200018
158    31807199705
159    31807151989
Name: Реестровый номер закупки, dtype: object

In [17]:
i = 1
pause = 1
for order in orders223:
    tmpdf = get_win_info_233(order, pause)
    if tmpdf is not None:
        df = pd.concat([df, tmpdf])
        print(f'{order} - ok! {i} of {len(orders44)} is done')
    else:
        print(f'{order} - checked. {i} of {len(orders44)} is done')
    i += 1
    time.sleep(1)

31807326700 - ok! 1 of 159 is done
31807270424 - ok! 2 of 159 is done
31807200018 - ok! 3 of 159 is done
31807199705 - ok! 4 of 159 is done
31807151989 - ok! 5 of 159 is done
31807219596 - ok! 6 of 159 is done
31807153786 - ok! 7 of 159 is done
31807147180 - ok! 8 of 159 is done
31807002991 - ok! 9 of 159 is done


### Присоединяем полученные реультаты к данным:

In [18]:
test = pd.merge(data, df,  how='left', on='Реестровый номер закупки')
test

,Реестровый номер закупки,Закупки по,"Способ определения поставщика, подрядной организации (размещения закупки)",Наименование закупки,type,Начальная (максимальная) цена контракта,Наименование Заказчика,Дата размещения,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Идентификационный код закупки,победитель,цена контракта после снижения,номер контракта
0,0349100004618000225,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, вывозу и обезврежи...",med,515100.00,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,Нет ограничений,2018-12-27,2019-01-21,'181511200012851120100100052180000000',"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СОРЭ...",512524,1511200012819000036
1,0349100004618000224,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по погрузке, транспортированию ...",med,879840.00,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-27,"Субъектам малого предпринимательства, социальн...",2018-12-27,2019-01-21,'181511200012851120100100052160000000',"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",875441,1511200012819000042
2,0349100004618000228,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по транспортированию и размещен...,med,49183.20,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-28,Нет ограничений,2018-12-28,2019-01-23,'181511200012851120100100052170000000',"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГРАН...",49183.2,1511200012819000029
3,0149200002318006192,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по транспортированию и обезвреж...,med,1115000.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-12-25,Нет ограничений,2018-12-25,2019-01-11,'182519080011451900100100662030000000',NaN,NaN,NaN
4,0349100021118000118,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по вывозу и утилизации медицинс...,med,1086758.87,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,2018-12-19,"Субъектам малого предпринимательства, социальн...",2018-12-19,2018-12-28,'181519005315951900100101181163821000',"Общество с ограниченной ответственностью ""Сорэ...",516211,1519005315919000016
5,0149200002318005842,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по вывозу и обезвреживанию меди...,med,590600.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-12-07,Нет ограничений,2018-12-07,2018-12-27,'182519214003051900100100050130000000',ООО 'СорэксМед',572882,2519214003019000009
6,0349200022018000059,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по вывозу и обезвреживанию меди...,med,310464.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-12-18,Нет ограничений,2018-12-18,2018-12-28,'182519006077351900100100100140000000',Общества с ограниченной ответственностью 'Сорэ...,308880,2519006077319000008
7,0149200002318005863,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по вывозу и обезвреживанию меди...,med,1242360.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-12-10,Нет ограничений,2018-12-10,2018-12-21,'182510890002051080100100200033822000',"ЗАО ""Мурманский универсальный завод""",689510,2510890002019000002
8,0149200002318005050,44-ФЗ,Иной однолотовый способ 44-ФЗ,Оказание услуг по транспортированию и обезвреж...,med,684000.00,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-11-15,Нет ограничений,2018-11-15,2018-12-04,'182519150067451900100100090030000000',"ООО ""ОРКО-ЦЕНТР""",147060,2519150067418000350
9,0149200002318005280,44-ФЗ,Иной однолотовый способ 44-ФЗ,"Оказание услуг по планово-регулярному вывозу, ...",med,1101296.25,ГОСУДАРСТВЕННОЕ ОБЛАСТНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ...,2018-11-22,Нет ограничений,2018-11-22,2018-12-04,'182519040415051900100100040030000000',ООО 'СорэксМед',1.09579e+06,2519040415018000092


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 15 columns):
Реестровый номер закупки                                                     170 non-null object
Закупки по                                                                   170 non-null object
Способ определения поставщика, подрядной организации (размещения закупки)    170 non-null object
Наименование закупки                                                         170 non-null object
type                                                                         170 non-null object
Начальная (максимальная) цена контракта                                      170 non-null float64
Наименование Заказчика                                                       170 non-null object
Дата размещения                                                              170 non-null datetime64[ns]
Особенности размещения заказа\ Преимущества (44-ФЗ)                          170 non-null object
Дата начала п

### Обнаружил задублированные данные. Проверяю и удаляю дубли закупок:

In [20]:
len(test['Реестровый номер закупки'].unique())

167

In [21]:
len(test)

170

In [22]:
test.drop_duplicates(subset='Реестровый номер закупки',  inplace=True)

In [23]:
len(test['Реестровый номер закупки'].unique()) == len(test)

True

###  Удаляем несостоявшиеся закупки, где победитель не был выявлен:

In [24]:
test = test.loc[test['победитель'].notna()]
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 168
Data columns (total 15 columns):
Реестровый номер закупки                                                     146 non-null object
Закупки по                                                                   146 non-null object
Способ определения поставщика, подрядной организации (размещения закупки)    146 non-null object
Наименование закупки                                                         146 non-null object
type                                                                         146 non-null object
Начальная (максимальная) цена контракта                                      146 non-null float64
Наименование Заказчика                                                       146 non-null object
Дата размещения                                                              146 non-null datetime64[ns]
Особенности размещения заказа\ Преимущества (44-ФЗ)                          146 non-null object
Дата начала п

### Заменяем некорректные значения цены после снижения (ошибка закупщика):

In [25]:
test.applymap(np.isreal)

,Реестровый номер закупки,Закупки по,"Способ определения поставщика, подрядной организации (размещения закупки)",Наименование закупки,type,Начальная (максимальная) цена контракта,Наименование Заказчика,Дата размещения,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Идентификационный код закупки,победитель,цена контракта после снижения,номер контракта
0,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
1,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
2,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
4,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
5,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
6,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
7,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
8,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
9,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False
10,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False


In [26]:
floatprice = pd.to_numeric(test['цена контракта после снижения'], errors="coerce").isna()
test[floatprice]

,Реестровый номер закупки,Закупки по,"Способ определения поставщика, подрядной организации (размещения закупки)",Наименование закупки,type,Начальная (максимальная) цена контракта,Наименование Заказчика,Дата размещения,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Идентификационный код закупки,победитель,цена контракта после снижения,номер контракта
161,31807219596,223-ФЗ,Закупка у единственного поставщика (подрядчика...,Услуги по размещению отходов производства и ...,?,500000.0,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""МУРМАНСКАЯ ТЭЦ""",2018-11-29,Нет ограничений,2018-11-29,NaT,NaN,"ООО ""ОРКО-инвест""","цена размещения 1 м3 отходов III, IV классов о...",31807219596-01


In [27]:
 test.loc[(pd.to_numeric(test['цена контракта после снижения'], errors="coerce").isna()), 'цена контракта после снижения'] = test.loc[(pd.to_numeric(test['цена контракта после снижения'], errors="coerce").isna()), 'Начальная (максимальная) цена контракта']

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 168
Data columns (total 15 columns):
Реестровый номер закупки                                                     146 non-null object
Закупки по                                                                   146 non-null object
Способ определения поставщика, подрядной организации (размещения закупки)    146 non-null object
Наименование закупки                                                         146 non-null object
type                                                                         146 non-null object
Начальная (максимальная) цена контракта                                      146 non-null float64
Наименование Заказчика                                                       146 non-null object
Дата размещения                                                              146 non-null datetime64[ns]
Особенности размещения заказа\ Преимущества (44-ФЗ)                          146 non-null object
Дата начала п

In [29]:
contracts44 = test.loc[(test['Закупки по'] == '44-ФЗ')]['номер контракта']
contracts44[contracts44.isna()]

Series([], Name: номер контракта, dtype: object)

In [30]:
for contruct in list(contracts44):
    get_contruct44(contruct)
    print(f'contract №{contruct} is successfully downloaded!!')
    time.sleep(random.randint(1,5))

contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downloaded!!
contract №10 is successfully downl

In [37]:
contracts223 = test.loc[(test['Закупки по'] == '223-ФЗ')]['Реестровый номер закупки']
contracts223[contracts223.isna()]

Series([], Name: Реестровый номер закупки, dtype: object)

In [64]:
i = 1
for contract in list(contracts223):
    get_contruct233(contract)
    print(f'contract №{contract} is successfully downloaded! {i} of {len(contracts223)}')
    i += 1
    time.sleep(random.randint(1,5))

contract №31807326700 is successfully downloaded! 1 of 7
contract №31807270424 is successfully downloaded! 2 of 7
contract №31807151989 is successfully downloaded! 3 of 7
contract №31807219596 is successfully downloaded! 4 of 7
contract №31807153786 is successfully downloaded! 5 of 7
contract №31807147180 is successfully downloaded! 6 of 7
contract №31807002991 is successfully downloaded! 7 of 7


In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 168
Data columns (total 15 columns):
Реестровый номер закупки                                                     146 non-null object
Закупки по                                                                   146 non-null object
Способ определения поставщика, подрядной организации (размещения закупки)    146 non-null object
Наименование закупки                                                         146 non-null object
type                                                                         146 non-null object
Начальная (максимальная) цена контракта                                      146 non-null float64
Наименование Заказчика                                                       146 non-null object
Дата размещения                                                              146 non-null datetime64[ns]
Особенности размещения заказа\ Преимущества (44-ФЗ)                          146 non-null object
Дата начала п

In [75]:
directory = '/Murmansk17-18/'
filename = now() + '.csv'
test.to_csv(filename, index=False)